# Equal Weight S&P Screener

## Table of Contents

## Project Overview

S&P 500 is the world's most popular stock market index. Essentially the 500 biggest U.S Companies

- Many investment funds are benchmarked to the S&P 500 (They seek to replicate the performance of this index)
- The S&P 500 is market cap weighted

This project aims to build an alternative version of this index where each company has the same weighting.
The specific goal is to create a python script that will accept the value of your portfolio and tell you how many shares of each S&P 500 constituent you should purchase to get an equal-weight verision of the index fund

## Libraries

In [2]:
import numpy as np
import pandas as pd
import requests
import xlsxwriter
import math

## Import List of Stocks
A list of ticker symbols in the S&P 500

In [16]:
tickers = pd.read_csv("sp_500_stocks.csv")

## Setting Up the API
Connecting to the IEX Cloud API. This is the data provider that we will be using throughout these projects.

In [9]:
IEX_CLOUD_API_TOKEN = 'Tpk_059b97af715d417d9f49f50b51b1c448'

In [19]:
def api_url(tick):
    return f'https://sandbox.iexapis.com/stable/stock/{tick}/quote?token={IEX_CLOUD_API_TOKEN}'
data = requests.get(api_url('AAPL'))
data

<Response [200]>

## Add API data to stocks DataFrame

In [23]:
cols = ['Ticker', 'Price','Market Capitalisation', 'Number Of Shares to Buy']
df = pd.DataFrame(columns=cols)

,Ticker,Price,Market Capitalisation,Number Of Shares to Buy


In [25]:
# Testing for the first 5 companies
for tick in tickers['Ticker'][:5]:
    data = requests.get(api_url(tick)).json()
    df = df.append(pd.Series([
        tick,
        data['latestPrice'],
        data['marketCap'],
        np.nan
    ], index=cols), ignore_index=True)

,Ticker,Price,Market Capitalisation,Number Of Shares to Buy
0,A,161.03,49858806226,NaN
1,AAL,18.80,11945781294,NaN
2,AAP,249.49,15377434195,NaN
3,AAPL,183.30,2944131785366,NaN
4,ABBV,138.19,248509675091,NaN


## Using Batch API Calls to improve performance

In [27]:
# Function sourced from 
# https://stackoverflow.com/questions/312443/how-do-you-split-a-list-into-evenly-sized-chunks
def chunks(lst, n):
    """Yield successive n-sized chunks from lst."""
    for i in range(0, len(lst), n):
        yield lst[i:i + n]

In [33]:
symbol_groups = list(chunks(tickers['Ticker'], 100))
symbol_strings = []
for i in range(0, len(symbol_groups)):
    symbol_strings.append(','.join(symbol_groups[i]))

In [37]:
for symbol_string in symbol_strings:
    batch_api_call_url = f'https://sandbox.iexapis.com/stable/stock/market/batch/?types=quote&symbols={symbol_string}&token={IEX_CLOUD_API_TOKEN}'
    data = requests.get(batch_api_call_url).json()
    for symbol in symbol_string.split(','):
        df = df.append(pd.Series(
            [
                symbol, 
                data[symbol]['quote']['latestPrice'], 
                data[symbol]['quote']['marketCap'], 
                'N/A'
            ], index = cols), ignore_index = True)

df

,Ticker,Price,Market Capitalisation,Number Of Shares to Buy
0,A,161.03,49858806226,NaN
1,AAL,18.80,11945781294,NaN
2,AAP,249.49,15377434195,NaN
3,AAPL,183.30,2944131785366,NaN
4,ABBV,138.19,248509675091,NaN
...,...,...,...,...
505,YUM,143.94,41077932186,N/A
506,ZBH,131.22,27084788326,N/A
507,ZBRA,621.23,33498302471,N/A
508,ZION,64.14,10263997196,N/A


## Calculating the number of shares to buy

In [46]:
PORTFOLIO_SIZE = 10000000 # 10 Million

allocation = PORTFOLIO_SIZE / len(df)

for i, row in df.iterrows():
    df.loc[i, "Number Of Shares to Buy"] = math.floor(allocation / row['Price'])

In [47]:
df

,Ticker,Price,Market Capitalisation,Number Of Shares to Buy
0,A,161.03,49858806226,121
1,AAL,18.80,11945781294,1042
2,AAP,249.49,15377434195,78
3,AAPL,183.30,2944131785366,106
4,ABBV,138.19,248509675091,141
...,...,...,...,...
505,YUM,143.94,41077932186,136
506,ZBH,131.22,27084788326,149
507,ZBRA,621.23,33498302471,31
508,ZION,64.14,10263997196,305
